# Recommender System based on Review Book in E-commerce Using word2vec Algorithm


1. Yolanda R.M Manurung (12S17017)
2. Stella Sitinjak      (12S17027)
3. Ekis Naomi Lasma     (12S17059)


### Dataset
Data yang digunakan pada proyek ini berasal dari kaggle (https://www.kaggle.com/gapple/goodreads-book-reviews-20k)

In [1]:
%matplotlib inline
# import sys
# reload(sys)
# sys.setdefaultencoding('utf-8')
import re
import csv
import ast
import operator
import pandas as pd
import seaborn as sns
import string
from string import digits
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import sent_tokenize, LineTokenizer, RegexpTokenizer

from collections import Counter
from matplotlib import cm
from itertools import cycle, islice
import matplotlib as mpl
import matplotlib.pyplot as plt
from subprocess import check_output
from wordcloud import WordCloud


from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

from pandas import *
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn import linear_model
import numpy as np

In [2]:
review = pd.read_csv('all_data.csv')
print ('Jumlah data =', len(review))
review.head()

Jumlah data = 21559


,book_title,Book_series,book_series_url,book_image,book_image_url,book_rating,book_author,author_url,genre,reviewer_name,reviewer_url,reviewer_image,reviewer_image_url,review,ID
0,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",emma,https://www.goodreads.com/user/show/32879029-emma,https://images.gr-assets.com/users/1520568130p...,https://www.goodreads.com/user/show/32879029-emma,202.\nThat’s the number of times that the word...,1
1,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Em (RunawayWithDreamthieves),https://www.goodreads.com/user/show/42130592-e...,https://images.gr-assets.com/users/1538507799p...,https://www.goodreads.com/user/show/42130592-e...,I need a black silk robe and a balcony to wear...,2
2,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Cait • A Page with a View,https://www.goodreads.com/user/show/47412532-c...,https://images.gr-assets.com/users/1523134490p...,https://www.goodreads.com/user/show/47412532-c...,THIS BOOK TOTALLY WRECKED ME. In the best poss...,3
3,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Natalie Monroe,https://www.goodreads.com/user/show/6688207-na...,https://images.gr-assets.com/users/1339730817p...,https://www.goodreads.com/user/show/6688207-na...,"3.75 stars\n""Rhys stepped off the stairs and t...",4
4,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Emily May,https://www.goodreads.com/user/show/4622890-em...,https://images.gr-assets.com/users/1528390751p...,https://www.goodreads.com/user/show/4622890-em...,I wish so much that I could tell you I loved t...,5


In [3]:
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21559 entries, 0 to 21558
Data columns (total 15 columns):
book_title            21559 non-null object
Book_series           8780 non-null object
book_series_url       8780 non-null object
book_image            21557 non-null object
book_image_url        21557 non-null object
book_rating           21559 non-null float64
book_author           21559 non-null object
author_url            21559 non-null object
genre                 21438 non-null object
reviewer_name         21534 non-null object
reviewer_url          21534 non-null object
reviewer_image        21534 non-null object
reviewer_image_url    21534 non-null object
review                20817 non-null object
ID                    21559 non-null int64
dtypes: float64(1), int64(1), object(13)
memory usage: 2.5+ MB


In [4]:
review.columns

Index(['book_title', 'Book_series', 'book_series_url', 'book_image',
       'book_image_url', 'book_rating', 'book_author', 'author_url', 'genre',
       'reviewer_name', 'reviewer_url', 'reviewer_image', 'reviewer_image_url',
       'review', 'ID'],
      dtype='object')

In [5]:
review.isna().sum()

book_title                0
Book_series           12779
book_series_url       12779
book_image                2
book_image_url            2
book_rating               0
book_author               0
author_url                0
genre                   121
reviewer_name            25
reviewer_url             25
reviewer_image           25
reviewer_image_url       25
review                  742
ID                        0
dtype: int64

Data Cleaning and Preperation

In [6]:
review.shape

(21559, 15)

Drop Null Value

In [7]:
df = pd.DataFrame(review[['book_title','book_rating','genre','review']])

In [8]:
review.shape

(21559, 15)

In [9]:
data_train, data_test = train_test_split(review, test_size=0.2)
data_train_copy = data_train
print("%d items in training data, %d in test data" % (len(data_train), len(data_test)))

17247 items in training data, 4312 in test data


In [10]:
df  = pd.DataFrame(review[['book_title','book_rating','genre','review']])

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21559 entries, 0 to 21558
Data columns (total 4 columns):
book_title     21559 non-null object
book_rating    21559 non-null float64
genre          21438 non-null object
review         20817 non-null object
dtypes: float64(1), object(3)
memory usage: 673.8+ KB


In [12]:
df.describe()

,book_rating
count,21559.000000
mean,4.017090
std,0.316388
min,0.000000
25%,3.850000
50%,4.050000
75%,4.230000
max,5.000000


In [13]:
df['review'].head(20)

0     202.\nThat’s the number of times that the word...
1     I need a black silk robe and a balcony to wear...
2     THIS BOOK TOTALLY WRECKED ME. In the best poss...
3     3.75 stars\n"Rhys stepped off the stairs and t...
4     I wish so much that I could tell you I loved t...
5     I’m wrecked. Because a Court of Wings and Ruin...
6     Yay! Another one of my books won 2017 GR award...
7     Congratulations-winner of Best Young Adult Fan...
8     In these early days after the release, I'll sa...
9     Glorious, thrilling and completely enthralling...
10    *Do I need to remind you to be civil? No, I do...
11    "And will you come with me? On this adventure ...
12    When the book you're highly anticipating has n...
13    BIBLIOBABBLE- https://youtu.be/y5UVdj8avvs (re...
14    I. CAN'T. HANDLE. THIS. WHAT???\nRTC\n--------...
15    Reread x 2.\nMy heart is in pain and full of l...
16    You know you're a dedicated fan when you mark ...
17    4.25/5 stars\n"Remember that you are a wol

Remove Punctuation

In [14]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [15]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

df['book_title'] = df['book_title'].apply(lambda x: remove_punct(x))
df['review'] = df['review'].apply(lambda x: remove_punct(x))
df.head(10)

TypeError: 'float' object is not iterable

Case Folding

In [16]:
df['review'] = df['review'].str.lower()
df['book_title'] = df['book_title'].str.lower()

df.head()

,book_title,book_rating,genre,review
0,a court of wings and ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",202.\nthat’s the number of times that the word...
1,a court of wings and ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",i need a black silk robe and a balcony to wear...
2,a court of wings and ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",this book totally wrecked me. in the best poss...
3,a court of wings and ruin,4.51,"Fantasy, Romance, Young Adult, New Adult","3.75 stars\n""rhys stepped off the stairs and t..."
4,a court of wings and ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",i wish so much that i could tell you i loved t...


Stopwords Removing

In [17]:
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [18]:
cachedStopWords = set(stopwords.words("english"))
for i in range (len(df)):
    meaningful = df['review'].iloc[i]
    meaningful = " ".join([word for word in meaningful.split() if word not in cachedStopWords])
    df['review'].iloc[i]=meaningful
print ('Data trainstop words removing... ')
df.head(20)

C:\Users\ITD-STU\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


AttributeError: 'float' object has no attribute 'split'

In [38]:
df.to_csv("dfReview.csv")

In [39]:
df = pd.read_csv("dfReview.csv")

In [40]:
import warnings
warnings.filterwarnings("ignore")

In [41]:
import pandas as pd
import re
import gensim
from gensim.models import Word2Vec
import numpy as np
from scipy import linalg

In [42]:
data = pd.read_csv("./all_data.csv")
data.head()

,book_title,Book_series,book_series_url,book_image,book_image_url,book_rating,book_author,author_url,genre,reviewer_name,reviewer_url,reviewer_image,reviewer_image_url,review,ID
0,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",emma,https://www.goodreads.com/user/show/32879029-emma,https://images.gr-assets.com/users/1520568130p...,https://www.goodreads.com/user/show/32879029-emma,202.\nThat’s the number of times that the word...,1
1,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Em (RunawayWithDreamthieves),https://www.goodreads.com/user/show/42130592-e...,https://images.gr-assets.com/users/1538507799p...,https://www.goodreads.com/user/show/42130592-e...,I need a black silk robe and a balcony to wear...,2
2,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Cait • A Page with a View,https://www.goodreads.com/user/show/47412532-c...,https://images.gr-assets.com/users/1523134490p...,https://www.goodreads.com/user/show/47412532-c...,THIS BOOK TOTALLY WRECKED ME. In the best poss...,3
3,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Natalie Monroe,https://www.goodreads.com/user/show/6688207-na...,https://images.gr-assets.com/users/1339730817p...,https://www.goodreads.com/user/show/6688207-na...,"3.75 stars\n""Rhys stepped off the stairs and t...",4
4,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Emily May,https://www.goodreads.com/user/show/4622890-em...,https://images.gr-assets.com/users/1528390751p...,https://www.goodreads.com/user/show/4622890-em...,I wish so much that I could tell you I loved t...,5


# Data Preprocessing

In [43]:
data.isnull().sum()

book_title                0
Book_series           12779
book_series_url       12779
book_image                2
book_image_url            2
book_rating               0
book_author               0
author_url                0
genre                   121
reviewer_name            25
reviewer_url             25
reviewer_image           25
reviewer_image_url       25
review                  742
ID                        0
dtype: int64

In [44]:
data.dtypes

book_title             object
Book_series            object
book_series_url        object
book_image             object
book_image_url         object
book_rating           float64
book_author            object
author_url             object
genre                  object
reviewer_name          object
reviewer_url           object
reviewer_image         object
reviewer_image_url     object
review                 object
ID                      int64
dtype: object

In [45]:
data['Book_series'] = data['Book_series'].fillna(data['Book_series'].mode().iloc[0])
data['book_series_url'] = data['book_series_url'].fillna(data['book_series_url'].mode().iloc[0])
data['book_image'] = data['book_image'].fillna(data['book_image'].mode().iloc[0])
data['book_image_url'] = data['book_image_url'].fillna(data['book_image_url'].mode().iloc[0])
data['genre'] = data['genre'].fillna(data['genre'].mode().iloc[0])
data['reviewer_name'] = data['reviewer_name'].fillna(data['reviewer_name'].mode().iloc[0])
data['reviewer_url'] = data['reviewer_url'].fillna(data['reviewer_url'].mode().iloc[0])
data['reviewer_image'] = data['reviewer_image'].fillna(data['reviewer_image'].mode().iloc[0])
data['reviewer_image_url'] = data['reviewer_image_url'].fillna(data['reviewer_image_url'].mode().iloc[0])
data['review'] = data['review'].fillna(data['review'].mode().iloc[0])

In [46]:
data.isnull().sum()

book_title            0
Book_series           0
book_series_url       0
book_image            0
book_image_url        0
book_rating           0
book_author           0
author_url            0
genre                 0
reviewer_name         0
reviewer_url          0
reviewer_image        0
reviewer_image_url    0
review                0
ID                    0
dtype: int64

In [47]:
data.duplicated().sum()

0

In [48]:
data['book_title'].duplicated().sum()

20837

In [49]:
data['book_title'].drop_duplicates(keep=False,inplace=True)
data.head()

,book_title,Book_series,book_series_url,book_image,book_image_url,book_rating,book_author,author_url,genre,reviewer_name,reviewer_url,reviewer_image,reviewer_image_url,review,ID
0,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",emma,https://www.goodreads.com/user/show/32879029-emma,https://images.gr-assets.com/users/1520568130p...,https://www.goodreads.com/user/show/32879029-emma,202.\nThat’s the number of times that the word...,1
1,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Em (RunawayWithDreamthieves),https://www.goodreads.com/user/show/42130592-e...,https://images.gr-assets.com/users/1538507799p...,https://www.goodreads.com/user/show/42130592-e...,I need a black silk robe and a balcony to wear...,2
2,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Cait • A Page with a View,https://www.goodreads.com/user/show/47412532-c...,https://images.gr-assets.com/users/1523134490p...,https://www.goodreads.com/user/show/47412532-c...,THIS BOOK TOTALLY WRECKED ME. In the best poss...,3
3,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Natalie Monroe,https://www.goodreads.com/user/show/6688207-na...,https://images.gr-assets.com/users/1339730817p...,https://www.goodreads.com/user/show/6688207-na...,"3.75 stars\n""Rhys stepped off the stairs and t...",4
4,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Emily May,https://www.goodreads.com/user/show/4622890-em...,https://images.gr-assets.com/users/1528390751p...,https://www.goodreads.com/user/show/4622890-em...,I wish so much that I could tell you I loved t...,5


In [50]:
coba = data["review"][0]
coba

"202.\nThat’s the number of times that the word “mate,” in any of its hellish forms, is used in this book.\nTWO.\nHUNDRED.\nAND.\nTWO.\nhttps://emmareadstoomuch.wordpress.co...\nThat statistic alone should be enough to justify this one star rating. If you feel satisfied enough to just remove yourself from this review, run with that feeling. Because I have, like. Seven pages of sh*t to complain about.\nThat’s if I’m efficient.\nAnd I’ve never been an efficient reviewer. Once. In my life. I counted two hundred (and two) uses of a word over seven hundred pages. That shouldn’t scream “rational and in control of her passionate fury” to you.\nSo buckle up, buttercup. We’ve got some categories to run through.\nDISCLAIMER : This review is teeming with spoilers. It is also X-rated, due to the strength of my anger and the copious sex scenes in the source material.\nI HATE THESE CHARACTERS\nRhys makes me cringe harder than Renee Zellweger in Bridget Jones’ Diary. (At least that movie has both Col

In [51]:
review = coba.replace(",", "").replace("\n", "").replace("\r", "").replace("---", "").replace('|','').replace("!","").replace("//", "").replace("?","").replace("    ", " ").replace("  "," ").casefold()
review

"202.that’s the number of times that the word “mate” in any of its hellish forms is used in this book.two.hundred.and.two.https:emmareadstoomuch.wordpress.co...that statistic alone should be enough to justify this one star rating. if you feel satisfied enough to just remove yourself from this review run with that feeling. because i have like. seven pages of sh*t to complain about.that’s if i’m efficient.and i’ve never been an efficient reviewer. once. in my life. i counted two hundred (and two) uses of a word over seven hundred pages. that shouldn’t scream “rational and in control of her passionate fury” to you.so buckle up buttercup. we’ve got some categories to run through.disclaimer : this review is teeming with spoilers. it is also x-rated due to the strength of my anger and the copious sex scenes in the source material.i hate these charactersrhys makes me cringe harder than renee zellweger in bridget jones’ diary. (at least that movie has both colin firth and hugh grant to distrac

In [52]:
review_word = review.split(" ")
review_word

['202.that’s',
 'the',
 'number',
 'of',
 'times',
 'that',
 'the',
 'word',
 '“mate”',
 'in',
 'any',
 'of',
 'its',
 'hellish',
 'forms',
 'is',
 'used',
 'in',
 'this',
 'book.two.hundred.and.two.https:emmareadstoomuch.wordpress.co...that',
 'statistic',
 'alone',
 'should',
 'be',
 'enough',
 'to',
 'justify',
 'this',
 'one',
 'star',
 'rating.',
 'if',
 'you',
 'feel',
 'satisfied',
 'enough',
 'to',
 'just',
 'remove',
 'yourself',
 'from',
 'this',
 'review',
 'run',
 'with',
 'that',
 'feeling.',
 'because',
 'i',
 'have',
 'like.',
 'seven',
 'pages',
 'of',
 'sh*t',
 'to',
 'complain',
 'about.that’s',
 'if',
 'i’m',
 'efficient.and',
 'i’ve',
 'never',
 'been',
 'an',
 'efficient',
 'reviewer.',
 'once.',
 'in',
 'my',
 'life.',
 'i',
 'counted',
 'two',
 'hundred',
 '(and',
 'two)',
 'uses',
 'of',
 'a',
 'word',
 'over',
 'seven',
 'hundred',
 'pages.',
 'that',
 'shouldn’t',
 'scream',
 '“rational',
 'and',
 'in',
 'control',
 'of',
 'her',
 'passionate',
 'fury”',
 'to'

In [53]:
review_vocab = set(review_word)
review_vocab

{'evil',
 'moment',
 'once',
 'amused',
 'do',
 'most',
 'i’m',
 'fury',
 'with',
 '“see',
 'personification',
 'romantic',
 'getting',
 'none',
 'actions',
 'she',
 'almost',
 'definitely',
 'take',
 'alone',
 'playing',
 'knew',
 'wanna',
 'at',
 'occupied',
 'feyre’s',
 'efficient',
 'place',
 'fast',
 'bi',
 'military',
 'awful.',
 'base',
 'derails',
 'battle',
 'material.i',
 'doll:',
 'thingsthe',
 'fangirls',
 'which',
 'and...other',
 '(i’m',
 'much',
 'powerful',
 'forgotten.',
 'times',
 'angry',
 'might',
 'in',
 'alive',
 'recognizes',
 'talk.plotline',
 'between',
 'tend',
 'passage.',
 'courageous.',
 'winnowing',
 'confronts',
 'flat',
 'read.it’s',
 'family',
 'scenes',
 'extraordinaire',
 'tucking',
 'guest',
 'smut.i’m',
 'redeem',
 'there',
 'my-real-family-is-the-one-i-made-not-the-one-i-was-born-with',
 'narratives.',
 'ways:',
 'well',
 'interesting',
 'nesta',
 'he’ll',
 'you.”“i',
 'saying',
 'ask',
 'have',
 'women.',
 'brains',
 'powers',
 '“species',
 'paint

In [54]:
review = re.sub(r'\ by.*? on', "", review)
review = re.sub(r"\d\.\d", "", review)
review = re.sub(r"\d+\ [a-z]+\.*\ \d+", "", review)
review = review.replace(".", " ")
review = re.sub(r'\  *', " ", review)
review

"202 that’s the number of times that the word “mate” in any of its hellish forms is used in this book two hundred and two https:emmareadstoomuch wordpress co that statistic alone should be enough to justify this one star rating if you feel satisfied enough to just remove yourself from this review run with that feeling because i have like seven pages of sh*t to complain about that’s if i’m efficient and i’ve never been an efficient reviewer once in my life i counted two hundred (and two) uses of a word over seven hundred pages that shouldn’t scream “rational and in control of her passionate fury” to you so buckle up buttercup we’ve got some categories to run through disclaimer : this review is teeming with spoilers it is also x-rated due to the strength of my anger and the copious sex scenes in the source material i hate these charactersrhys makes me cringe harder than renee zellweger in bridget jones’ diary (at least that movie has both colin firth and hugh grant to distract me from re

In [55]:
print(len(data['review']))

21559


In [56]:
datas = []
for i in range(0, 21559):
    datas.append(data["review"][i]. replace(",", "").replace("\n", "").replace("\r", "").replace("---", "").replace('|','').replace("!","").replace("//", "").replace("?","").replace("    ", " ").replace("  "," ").casefold())

In [57]:
review = []
for i in range(0, 21559):
    review.append(data["review"][i].replace(",", "").replace("\n", "").replace("\r", "").replace("---", "").replace('|','').replace("!","").replace("//", "").replace("?","").replace("    ", " ").replace("  "," ").casefold())

In [58]:
reviews = []
for i in review:
    review = re.sub(r'\ by.*? on', "", i)
    review = re.sub(r"\d\.\d", "", review)
    review = re.sub(r"\d+\ [a-z]+\.*\ \d+", "", review)
    review = review.replace(".", " ")
    review = re.sub(r'\  *', " ", review)
    reviews.append(review)

In [59]:
data["reviews"] = reviews
data["reviews"]

0        202 that’s the number of times that the word “...
1        i need a black silk robe and a balcony to wear...
2        this book totally wrecked me in the best possi...
3        5 stars"rhys stepped off the stairs and took m...
4        i wish so much that i could tell you i loved t...
                               ...                        
21554    no one ask me about the book bc i literally re...
21555    i started the harry potter audio books at the ...
21556    “do not pity the dead harry pity the living an...
21557    harry potter changed my life if i hadn’t read ...
21558    lo que lloré con este libro no tiene nombre to...
Name: reviews, Length: 21559, dtype: object

In [60]:
data["review"][8461]

"5 STARS\nFirst off, I would like to sincerely thank Netgalley for the rare and precious ARC! I don't get them often, so imagine my thrill when I saw this baby!\nI will start out by saying that I am a huge fan of Sandra Brown and her gritty romance murder mysteries. But I want you to know that I'm not going to slap a 5 star rating up there just because I love the author's previous books, and because I am grateful for receiving this early release. (even though I am! ...grateful that is.) I going to do it because she has yet again delivered a gripping and emotionally charged story that led me down the primrose path, thinking I had it all worked out and then once again pulling the rug out from under my feet.\nThis was a very unique and intriguing storyline. One that would make a gripping movie. The kind you can on get from SB.\nLoved the characters. Even though I'm not a fan of reporters or newscasters of any kind, Kerra Bailey still managed to get under my skin and provoke a bit of empat

In [61]:
print("Banyak baris book title: {0}".format(data["book_title"].shape))
print("Banyak baris rating: {0}".format(data["book_rating"].shape))
print("Banyak baris genre: {0}".format(data["genre"].shape))
print("Banyak baris review: {0}".format(data["review"].shape))
print("Banyak baris, kolom data: {0}".format(data.shape))

Banyak baris book title: (21559,)
Banyak baris rating: (21559,)
Banyak baris genre: (21559,)
Banyak baris review: (21559,)
Banyak baris, kolom data: (21559, 16)


In [62]:
data[["book_title", "book_rating", "genre", "review"]].head()

,book_title,book_rating,genre,review
0,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",202.\nThat’s the number of times that the word...
1,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",I need a black silk robe and a balcony to wear...
2,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",THIS BOOK TOTALLY WRECKED ME. In the best poss...
3,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult","3.75 stars\n""Rhys stepped off the stairs and t..."
4,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",I wish so much that I could tell you I loved t...


In [63]:
new = data[["ID", "book_title", "book_rating", "genre", "review"]]
new.to_csv("./parse_nlp.csv")

In [64]:
dataset = pd.read_csv("./parse_nlp.csv", delimiter=",")
dataset

,Unnamed: 0,ID,book_title,book_rating,genre,review
0,0,1,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",202.\nThat’s the number of times that the word...
1,1,2,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",I need a black silk robe and a balcony to wear...
2,2,3,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",THIS BOOK TOTALLY WRECKED ME. In the best poss...
3,3,4,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult","3.75 stars\n""Rhys stepped off the stairs and t..."
4,4,5,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",I wish so much that I could tell you I loved t...
...,...,...,...,...,...,...
21554,21554,21555,Harry Potter and the deathly hallows,4.62,"Fantasy, Young Adult, Fiction",no one ask me about the book bc i literally re...
21555,21555,21556,Harry Potter and the deathly hallows,4.62,"Fantasy, Young Adult, Fiction",I started the Harry Potter audio books at the ...
21556,21556,21557,Harry Potter and the deathly hallows,4.62,"Fantasy, Young Adult, Fiction","“Do not pity the dead, Harry. Pity the living,..."
21557,21557,21558,Harry Potter and the deathly hallows,4.62,"Fantasy, Young Adult, Fiction",Harry Potter changed my life. If I hadn’t read...


## Tokenization

In [65]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataset, test_size = 0.2, random_state = 0)

In [66]:
len(train['book_title'].unique())

722

In [67]:
from nltk.tokenize import SpaceTokenizer
sTokenizer = SpaceTokenizer()
review_tokens = []
for i in train['review']:
    review_tokens.append(sTokenizer.tokenize(i))

In [68]:
train['review_tokens'] = review_tokens
train['review_tokens']

20663    [This, is, probably, the, second-to-last, wors...
14559    [4,5, Sterne!\n"Man, kann, sich, nicht, bis, ü...
548      [I, was, very, disappointed, with, this, book....
9569     [Enjoyed, this, one, a, lot., This, series, ke...
6484     [Sometimes, love, needs, a, second, chance, be...
                               ...                        
13123    [What's, the, worst, memory, you, have?, An, a...
19648    [IT’S, 2018, AND, AUTHORS, ARE, STILL, BASING,...
9845     [Book, 3, in, The, Stranje, House, series, did...
10799    [Obviously, very, influenced, by, Please, Kill...
2732     [Holy, crap,, this, book, was, INTENSE!, By, t...
Name: review_tokens, Length: 17247, dtype: object

In [69]:
vocab = []
for i in train['review_tokens']:
    vocab.append(set(i))

In [70]:
train['review_vocabs'] = vocab
train['review_vocabs']

20663    {This, as, there, elsewhere, ever, time, pride...
14559    {ist, erwarten,, spannend,, jetzt, wie, Hölle,...
548      {moment, lost., around, people,, written., tra...
9569     {This, I'm, around, who, time, help, him, hone...
6484     {another, do, second-chance, with, catnip., SO...
                               ...                        
13123    {psychiatrist,, explosive, ending., another, c...
19648    {shaming,, spend, I’ll, independent, leave, ex...
9845     {did, are, 3, Mr., Lady, Daneska, disappoint.,...
10799    {books, I'm, reflect, distill, most, them, the...
2732     {pacing, intensity, better, them, slowly, pull...
Name: review_vocabs, Length: 17247, dtype: object

## Word Embedding with Word2Vec

In [71]:
word2vec_sg = Word2Vec(sentences=train['review_vocabs'], size=100, window=5, min_count=1, workers=4, sg=1, iter=10)

In [72]:
word2vec_sg.wv.similarity(w1="great",w2="good")

0.14842752

In [73]:
w1 = ["great"]
word2vec_sg.wv.most_similar (positive=w1,topn=6)

[('Adam', 0.8090603947639465),
 ('play', 0.7536343336105347),
 ('Sounds', 0.7473795413970947),
 ('sold', 0.7439213991165161),
 ('More', 0.7285870313644409),
 ('others', 0.7230377197265625)]

In [74]:
w1 = ["good",'fun','great']
w2 = ['bad']
word2vec_sg.wv.most_similar (positive=w1,negative=w2,topn=10)

[('Some', 0.6533236503601074),
 ('th', 0.6502570509910583),
 ('obvious', 0.6484614610671997),
 ('frank,', 0.6419464349746704),
 ('Ganshert', 0.6409461498260498),
 ('Flaming', 0.6404467821121216),
 ('Prentiss', 0.6399499177932739),
 ('necessary', 0.6384363174438477),
 ('Kloos', 0.6374273300170898),
 ('Program', 0.6327961087226868)]

In [75]:
word2vec_sg.wv.similarity(w1="good",w2="good")

1.0

In [76]:
train['review_vocabs'][0]

{'DAEMATI',
 'YA???',
 'do',
 'with',
 'personification',
 'she',
 'alone',
 'CHARACTER',
 'place',
 'bi',
 'derails',
 'much',
 'powerful',
 'forgotten.',
 '“Ok,',
 '“Chicken',
 'recognizes',
 'tend',
 'winnowing',
 'scenes',
 'MEAN.',
 'ACOTAR,',
 'narratives.',
 'ask',
 'saying',
 'have',
 'brains',
 'it',
 'LIKE',
 'someone',
 '(a',
 'date',
 'triangle',
 'omg-he’s-just-trying-to-do-right-he-didn’t-know-any-better',
 'IT',
 'honor',
 'lately',
 'joy',
 'times,',
 'WHY.',
 'chapters',
 "it's",
 'call',
 'movement',
 'answer',
 'side',
 'performer',
 'ugly',
 'favorite',
 'will',
 'important',
 'smart',
 'guys,',
 'Rhysand',
 '“Got',
 'been',
 'people',
 'passionate',
 'said.',
 'memo.',
 'inconsistently',
 '“male.”\nWomen',
 'adolescence?',
 'Or,',
 'whatever.)',
 'Diary.',
 'Mor,',
 'prettiness',
 'laugh',
 'roommates',
 'sexuality',
 'characterized',
 'all.',
 'sense.',
 'ACTUAL',
 'plans',
 'dull',
 'straight,',
 'Feyre’s',
 'WITH.',
 'He',
 'mine,',
 'sickness.',
 'she’ll',
 'hi

In [77]:
word2vec_sg.save("word2vec_model")
w2v = Word2Vec.load("word2vec_model")

In [78]:
book_data = []
for key,book in enumerate(train['review_vocabs']):
    if len(book) == 0:
        print(book, key)
    feature_vector = np.zeros((100,), dtype="float32")
    nwords = len(book)
    for token in book:
        feature_vector = np.add(feature_vector, w2v[token])
    
    feature_vector = np.divide(feature_vector, nwords)
    book_data.append({'book': data['book_title'][key], 'tags':book, 'vector':feature_vector})

In [79]:
book_vector_df = pd.DataFrame(book_data,columns=['book', 'tags', 'vector'])
book_vector_df.head()

,book,tags,vector
0,A Court of Wings and Ruin,"{This, as, there, elsewhere, ever, time, pride...","[0.38423893, -0.24025282, 0.74989843, -0.57274..."
1,A Court of Wings and Ruin,"{ist, erwarten,, spannend,, jetzt, wie, Hölle,...","[0.1530361, -0.15838113, 0.31645805, -0.415542..."
2,A Court of Wings and Ruin,"{moment, lost., around, people,, written., tra...","[0.333776, -0.26613718, 0.68429095, -0.5962642..."
3,A Court of Wings and Ruin,"{This, I'm, around, who, time, help, him, hone...","[0.30164206, -0.34503633, 0.7117833, -0.701317..."
4,A Court of Wings and Ruin,"{another, do, second-chance, with, catnip., SO...","[0.28076056, -0.26724672, 0.61305034, -0.55715..."


In [80]:
book_vector_df.shape

(17247, 3)